In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import graphviz as gr
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
from causalimpact import CausalImpact
style.use("fivethirtyeight")


In [5]:
pip install causalimpact


  Using cached causalimpact-0.2.0-py3-none-any.whl (22 kB)
  Using cached statsmodels-0.11.0.tar.gz (15.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'
  Using cached causalimpact-0.1.5-py3-none-any.whl (17 kB)
  Using cached causalimpact-0.1.4.tar.gz (15 kB)
  Using cached statsmodels-0.8.0.tar.gz (9.5 MB)

  ERROR: Command errored out with exit status 1:
   command: 'C:\Python39\python.exe' 'C:\Users\user.DESKTOP-OMQ89VA\AppData\Local\Temp\pip-standalone-pip-28hxifta\__env_pip__.zip\pip' install --ignore-installed --no-user --prefix 'C:\Users\user.DESKTOP-OMQ89VA\AppData\Local\Temp\pip-build-env-8wkkk6fi\overlay' --no-warn-script-location --no-binary :none: --only-binary :none: -i https://pypi.org/simple -- setuptools wheel 'cython>=0.29.14' 'numpy==1.14.5; python_version=='"'"'3.5'"'"'' 'numpy==1.14.5; python_version=='"'"'3.6'"'"'' 'numpy==1.14.5; python_version>='"'"'3.7'"'"'' 'numpy==1.17.5; python_version>='"'"'3.8'"'"'' 'scipy>=1.0'
       cwd: None
  Complete output (23 lines):
  Ignoring numpy: markers 'python_version == "3.5"' don't match your environment
  Ignoring numpy: markers 'python_version == "3.6"' don't match your environment
    Using cached setuptools-57.4.0-py3-none-any.whl (819 kB)
    Using cached wheel-0.37.0-py2.py3-none-any.whl (35 kB)
    Using cached Cython-0.


Using legacy 'setup.py install' for causalimpact, since package 'wheel' is not installed.
Using legacy 'setup.py install' for statsmodels, since package 'wheel' is not installed.
    Running setup.py install for statsmodels: started
    Running setup.py install for statsmodels: finished with status 'error'



    ERROR: Command errored out with exit status 1:
     command: 'C:\Python39\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user.DESKTOP-OMQ89VA\\AppData\\Local\\Temp\\pip-install-fgdwzqpp\\statsmodels_85edf4f51d914c19b701ae29631fa217\\setup.py'"'"'; __file__='"'"'C:\\Users\\user.DESKTOP-OMQ89VA\\AppData\\Local\\Temp\\pip-install-fgdwzqpp\\statsmodels_85edf4f51d914c19b701ae29631fa217\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\user.DESKTOP-OMQ89VA\AppData\Local\Temp\pip-record-r6uscl_i\install-record.txt' --single-version-externally-managed --compile --install-headers 'C:\Python39\Include\statsmodels'
         cwd: C:\Users\user.DESKTOP-OMQ89VA\AppData\Local\Temp\pip-install-fgdwzqpp\stat

## Reading the dataframes from eda1.ipynb

In [2]:
%store -r features_df
%store -r target_df
%store -r clean_df

clean_df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
6,1,18.25,19.98,119.6,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,...,22.88,27.66,153.2,1606.0,0.1442,0.2576,0.3784,0.19320,0.3063,0.08368
7,1,13.71,20.83,90.2,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,...,17.06,28.14,110.6,897.0,0.1654,0.3682,0.2678,0.15560,0.3196,0.11510
10,1,16.02,23.24,102.7,797.8,0.08206,0.06669,0.03299,0.03323,0.1528,...,19.19,33.88,123.8,1150.0,0.1181,0.1551,0.1459,0.09975,0.2948,0.08452
11,1,15.78,17.89,103.6,781.0,0.09710,0.12920,0.09954,0.06606,0.1842,...,20.42,27.28,136.5,1299.0,0.1396,0.5609,0.3965,0.18100,0.3792,0.10480
13,1,15.85,23.95,103.7,782.7,0.08401,0.10020,0.09938,0.05364,0.1847,...,16.84,27.66,112.0,876.5,0.1131,0.1924,0.2322,0.11190,0.2809,0.06287


#### Split data into training and hold-out set

In [3]:
from sklearn.model_selection import train_test_split
x=clean_df.drop(columns=['diagnosis'])
y=clean_df[['diagnosis']]
# split the dataset to 80% train and 20%test set 
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.2,random_state=40)
print('xtrain',xtrain.shape,'ytrain',ytrain.shape)
print('xtest', xtest.shape,'ytest',ytest.shape)

xtrain (318, 30) ytrain (318, 1)
xtest (80, 30) ytest (80, 1)


In [4]:
# I. Create a causal model from the data and given graph.
model = CausalModel(
    data=data["clean_df"],
    treatment=data["radius_mean"],
    outcome=data["diagnosis"],
    graph=data["gml_graph"])

# II. Identify causal effect and return target estimands
identified_estimand = model.identify_effect()

# III. Estimate the target estimand using a statistical method.
estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.propensity_score_matching")

# IV. Refute the obtained estimate using multiple robustness checks.
refute_results = model.refute_estimate(identified_estimand, estimate,
                                       method_name="random_common_cause")

NameError: name 'CausalModel' is not defined